# Class II - An introduction to JuMP

*Los Alamos National Laboratory Grid Science Winter School, 2019*

Welcome! This tutorial will introduce you to the basics of JuMP. If you haven't yet, work through [Class I - An introduction  to Julia](Class%20I%20-%20An%20introduction%20to%20Julia.ipynb) first.

**Warning! This notebook is an introduction to JuMP v0.18. However, JuMP is currently undergoing a re-write. In the near future, some aspects of JuMP will change. But don't worry, the majority of this tutorial is still relevant in the future.**

This tutorial doesn't exist in isolation. Some other good resources for learning JuMP are
- [the Discourse forum](https://discourse.julialang.org/c/domain/opt)
- [JuMP documentation](http://www.juliaopt.org/JuMP.jl/0.18/)
- [JuMP examples](https://github.com/JuliaOpt/JuMP.jl/tree/release-0.18/examples)
- [Textbook: Julia Programming for Operations Research](http://www.chkwon.net/julia/)

Before we start, run the following magic sauce to install the required Julia packages and check that we're good to go.

In [5]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Excellent! Everything is good to go!


## The basics

First, load the JuMP package into your current environment.

In [6]:
using JuMP

Start building a JuMP model like so,

In [7]:
model = Model()
@variable(model, x)
@variable(model, y >= 0)
@variable(model, 1 <= z <= 2)
model

Feasibility problem with:
 * 0 linear constraints
 * 3 variables
Solver is default solver

### What's going on here?
`@variable(model, x)` does four things:
1. it adds an *optimization* variable to the model
2. it creates a *JuMP* variable that acts as a reference to the optimization variable in the model
3. it creates a *Julia* variable `x` that points to the JuMP variable
4. it stores a reference to the JuMP variable in the model with the name `:x`

In [8]:
model = Model()
@variable(model, x >= 1.414)

x

In [9]:
# x is a JuMP variable
typeof(x)

Variable

In [10]:
# We can bind the JuMP variable to a different Julia variable and set `x` to something else
y = x
x = 1

@show typeof(y)
@show typeof(x)

y

typeof(y) = Variable
typeof(x) = Int64


x

In [11]:
JuMP.getlowerbound(y)

1.414

In [12]:
model[:x]

x

In [13]:
model[:x] == y

true

### Other ways to create variables

We can also create arrays of JuMP variables.

In [14]:
model = Model()
@variable(model, x[i = 1:4] >= i)
x

4-element Array{Variable,1}:
 x[1]
 x[2]
 x[3]
 x[4]

The indices of the arrays don't have to be integers. They can be anything, like a string or a symbol.

In [15]:
model = Model()
@variable(model, x[i = 1:2, j = [:A, :B]] >= i)
x

x[i,j] ≥ … ∀ i ∈ {1,2}, j ∈ {A,B}

We can also add a condition to filter out some of the variables

In [16]:
model = Model()
@variable(model, x[i = 1:4, j = [:A, :B]; isodd(i)] >= i)
x

x[i,j] ≥ … ∀ i ∈ {1,2,3,4}, j ∈ {A,B} s.t. isodd(i)

What if I want to add two variables with the same name?

In [17]:
model = Model()
@variable(model, x >= 1)
@variable(model, x >= 2)

┌ Warning: A variable or constraint named x is already attached to this model. If creating variables programmatically, use the anonymous variable syntax x = @variable(m, [1:N], ...).
└ @ JuMP /Users/carleton/.julia/packages/JuMP/PbnIJ/src/JuMP.jl:852


x

### Quiz Question

What is the value of the following?

In [18]:
JuMP.getlowerbound(x)

2.0

### Anonymous variables

So far, we've only added *named* variables. This works great until we want to add two variables with the same name! To work around this isse, we can add *anonymous* variables.

In [19]:
model = Model()
x = @variable(model)
@show JuMP.getlowerbound(x)
x = @variable(model, lowerbound = 1)
@show JuMP.getlowerbound(x)
x = @variable(model, [i = 1:2], lowerbound = i)
@show JuMP.getlowerbound(x)

model[:x]  # This throws an error

JuMP.getlowerbound(x) = -Inf
JuMP.getlowerbound(x) = 1.0
JuMP.getlowerbound(x) = [1.0, 2.0]


KeyError: KeyError: key "No object with name x" not found

So what's the difference between the *named* and *anonymous* syntax? 

Well, `@variable(model, x >= 2)` is (roughly) equivalent to

In [20]:
model = Model()
x = model[:x] = @variable(model, lowerbound = 2, basename = "x")
model

Feasibility problem with:
 * 0 linear constraints
 * 1 variable
Solver is default solver

In [21]:
model = Model()
@variable(model, x >= 2)
model

Feasibility problem with:
 * 0 linear constraints
 * 1 variable
Solver is default solver

Although uneeded for this tutorial, we can also create binary and integer variables as follows:

In [22]:
model = Model()
@variable(model, x >= 1, Int)
@variable(model, y, Bin)
model

Feasibility problem with:
 * 0 linear constraints
 * 2 variables: 1 binary, 1 integer
Solver is default solver

### Constraints

Now that we've seen how to create variables, let's look at constraints. Much of the syntax should be familiar.

In [23]:
model = Model()
@variable(model, x >= 0)
@variable(model, y >= 0)

@constraint(model, c_less_than, 2x + y <= 1)
@constraint(model, c_greater_than, 2x + y >= 1)
@constraint(model, c_equal_to, 2x + y == 1)

model

Feasibility problem with:
 * 3 linear constraints
 * 2 variables
Solver is default solver

In [24]:
model[:c_equal_to]

2 x + y = 1

In [25]:
anonymous_constraint = @constraint(model, [i = 1:2], i * x <= y)

model

Feasibility problem with:
 * 5 linear constraints
 * 2 variables
Solver is default solver

### Objective Functions

Now let's look at the objective function.

In [26]:
model = Model()
@variable(model, x >= 0)

@objective(model, Min, 2x + 1)

model

Minimization problem with:
 * 0 linear constraints
 * 1 variable
Solver is default solver

In [27]:
model = Model()
@variable(model, x <= 2)

@objective(model, Max, 2x + 1)

model

Maximization problem with:
 * 0 linear constraints
 * 1 variable
Solver is default solver

### Solving a Model

Once we've formulated a model, the next step is to solve it. This requires a solver.

JuMP supports lots of different solvers. The [JuMP documentation](http://www.juliaopt.org/JuMP.jl/0.18/installation.html#getting-solvers) contains a list of the supported solvers and the types of problems each solver supports.

We're going to use two solvers in particular.

The first solver is the [GNU Linear Programming Kit (GLPK)](https://www.gnu.org/software/glpk/). This solver supports linear programs with continous variables.

GLPK is available via the [GLPKMathProgInterface.jl](https://github.com/JuliaOpt/GLPKMathProgInterface.jl) package.

In [28]:
using GLPKMathProgInterface
# Define an instance of the GLPK solver.
const GLPK_SOLVER = GLPKMathProgInterface.GLPKSolverLP()

┌ Info: Recompiling stale cache file /Users/carleton/.julia/compiled/v1.1/GLPKMathProgInterface/Y5bTM.ji for GLPKMathProgInterface [3c7084bd-78ad-589a-b5bb-dbd673274bea]
└ @ Base loading.jl:1184


GLPKMathProgInterface.GLPKInterfaceLP.GLPKSolverLP(false, :Simplex, Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}())

The second solver is the COIN-OR [Interior Point OPTimizer (Ipopt)](https://projects.coin-or.org/Ipopt). This solver supports nonlinear programs with continous variables.

Ipopt is available via the [Ipopt.jl](https://github.com/JuliaOpt/Ipopt.jl) package.

In [29]:
using Ipopt
const IPOPT_SOLVER = IpoptSolver(print_level=0)

┌ Info: Recompiling stale cache file /Users/carleton/.julia/compiled/v1.1/Ipopt/yMQMo.ji for Ipopt [b6b21f68-93f8-5de0-b562-5493be1d77c9]
└ @ Base loading.jl:1184


IpoptSolver(Tuple[(:print_level, 0)])

There are two ways to add a solver to a JuMP model:

In [30]:
model = Model()
JuMP.setsolver(model, GLPK_SOLVER)

# ... or ...

model = Model(solver = IPOPT_SOLVER)

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is Ipopt

If you try to solve an unsupported problem type, an error will be thrown:

In [31]:
model = Model(solver = GLPK_SOLVER)
@variable(model, 0 <= x <= π)
@NLobjective(model, Min, cos(x)^2)
solve(model)

MethodError: MethodError: no method matching NonlinearModel(::GLPKMathProgInterface.GLPKInterfaceLP.GLPKSolverLP)
Closest candidates are:
  NonlinearModel(!Matched::IpoptSolver) at /Users/carleton/.julia/packages/Ipopt/Iu7vT/src/MPB_wrapper.jl:38
  NonlinearModel(!Matched::Int64) at /Users/carleton/.julia/packages/MathProgBase/rVlFR/src/SolverInterface/SolverInterface.jl:29
  NonlinearModel() at /Users/carleton/.julia/packages/MathProgBase/rVlFR/src/SolverInterface/SolverInterface.jl:27

That error isn't very nice, but don't worry, the new version of JuMP has much nicer errors. This sort of thing shouldn't happen in the future.

In [32]:
JuMP.setsolver(model, IPOPT_SOLVER)
status = solve(model)

println("The solution status is: $(status)")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

The solution status is: Optimal


In [33]:
x_value = JuMP.getvalue(x)
obj_value = JuMP.getobjectivevalue(model)

println("      x | $(x_value)")
println("    π/2 | $(π/2)")
println("--------+----------------------")
println("cos²(x) | $(obj_value)")

      x | 1.5707963267948966
    π/2 | 1.5707963267948966
--------+----------------------
cos²(x) | 3.749399456654644e-33


## Example: Simple Economic Dispatch

*This example is adapted from a [tutorial given at the 2015 Grid Science Winter School](https://github.com/JuliaOpt/juliaopt-notebooks/blob/3110eddaf5effdecfee687739295bea05731ba33/notebooks/Dvorkin%20-%20Power%20systems%20-%20Economic%20dispatch%20and%20Unit%20commitment.ipynb).*

Economic dispatch (ED) is an optimization problem that minimizes the cost of supplying energy demand subject to operational constraints on power system assets. In its simplest instantiation, ED is a Linear Programing (LP) problem solved for an aggregated load and wind forecast and for a single infinitesimal moment. Mathematically, the ED problem can be written as follows:
$$
\min \sum_{i \in I} c^g_{i} \cdot g_{i} + c^w \cdot w,
$$
where $c_{i}$ and $g_{i}$ are the incremental cost ($\$/MWh$) and power output ($MW$) of the $i^{th}$ thermal generator, respectively, and $c^w$ and $w$ are the incremental cost ($\$/MWh$) and wind power injection ($MW$), respectively.

s.t.

<li> Minimum ($g^{\min}$) and maximum ($g^{\max}$) limits on power outputs of the thermal generators: </li>
$$
g^{\min}_{i} \leq g_{i} \leq g^{\max}_{i}.
$$
<li>Constraint on the wind power injection:</li>
$$
0 \leq w \leq w^f, 
$$
where $w$ and $w^f$ are the wind power injection and wind power forecast, respectively.

<li>Power balance constraint:</li>
$$
\sum_{i \in I} g_{i} + w = d, 
$$
where $d$ is the demand.

Further reading on ED models can be found in A. J. Wood, B. F. Wollenberg, and G. B. Sheblé, "Power Generation, Operation and Control", Wiley, 2013.

### JuMP Implementation of Economic Dispatch 

First, we need to load some plotting packages.  Note, compilation may take a few minutes.

In [34]:
using Interact
using Plots

┌ Info: Precompiling Interact [c601a237-2ae4-5e1e-952c-7a85b0c7eef1]
└ @ Base loading.jl:1186


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/1e41488c989fe7274a43be790c96b4757b8f9257-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/1e41488c989fe7274a43be790c96b4757b8f9257-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1186


Now, we define some problem data for a model with two thermal generators and one wind generator.

In [35]:
# Define some input data about the test system
# Maximum power output of generators
const GENERATION_MAX = [1000, 1000]
# Minimum power output of generators
const GENERATION_MIN = [0, 300]
# Incremental cost of generators 
const COST_GENERATION = [50, 100]
# Incremental cost of wind generators
const COST_WIND = 50
# Total demand
const DEMAND = 1500
# Wind forecast
const WIND_MAX = 200;

In the next cell, we create a Julia function that formulates and solves the economic dispatch problem.

In [36]:
"""
    solve_economic_dispatch(; cost_of_thermal::Vector, cost_of_wind)

Formulate and solve the economic dispatch problem given the cost of generation 
for the two thermal generators and the cost of wind generation.
"""
function solve_economic_dispatch(;
        cost_of_thermal = COST_GENERATION, 
        cost_of_wind = COST_WIND)

    economic_dispatch = Model(solver = GLPK_SOLVER) 
    
    # Define decision variables    
    @variables(economic_dispatch, begin
        g[i=1:2]  # Thermal generation (MW).
        w >= 0  # Wind power (MW).
    end)

    # Define the objective function
    @objective(economic_dispatch, Min, 
        sum(cost_of_thermal[i] * g[i] for i in 1:2) + cost_of_wind * w
    )

    # Define the constraint on the maximum and minimum power output of each generator.
    for i in 1:2
        @constraint(economic_dispatch, g[i] <= GENERATION_MAX[i])
        @constraint(economic_dispatch, g[i] >= GENERATION_MIN[i])
    end
    
    @constraints(economic_dispatch, begin
        # Define the constraint on the wind power injection
        w <= WIND_MAX
        # Define the power balance constraint
        sum(g[i] for i in 1:2) + w == DEMAND
    end)

    # Solve statement
    solve(economic_dispatch)
    
    # Return the optimal value of the objective function and its minimizers
    # as a NamedTuple.
    return (
        generation = getvalue(g), 
        wind_generation = getvalue(w),
        wind_spillage = WIND_MAX - getvalue(w),
        cost = getobjectivevalue(economic_dispatch)
    )
end

# Solve the economic dispatch problem
solution = solve_economic_dispatch()

println("Dispatch")
println("   Generators: ", solution.generation, " MW")
println("         Wind: ", solution.wind_generation, " MW")
println("Wind spillage: ", solution.wind_spillage, " MW") 
println("----------------------------------")
println("Total cost: \$", solution.cost)  

Dispatch
   Generators: [1000.0, 300.0] MW
         Wind: 200.0 MW
Wind spillage: 0.0 MW
----------------------------------
Total cost: $90000.0


### Economic dispatch with adjustable incremental costs

In the following exercise we introduce a manipulator to vary the cost of wind generation and observe its impact the total cost, dispatch of generators G1 and G2, utilization of available wind under different values of the incremental cost of generator G1.

In [37]:
@manipulate for cost_of_wind in COST_WIND .* (1:0.1:3.5)
    solutions = Any[]
    cost_of_g1 = COST_GENERATION[1] .* (0.5:0.01:3.0)
    for c_g1 in cost_of_g1
        # update the incremental cost of the first generator at every iteration
        solution = solve_economic_dispatch(
            cost_of_thermal = [c_g1, COST_GENERATION[2]],
            cost_of_wind = cost_of_wind
        )
        push!(solutions, solution)
    end
    
    # Plot the outputs
    plot(
        # Plot the total cost
        plot(cost_of_g1, [sol.cost for sol in solutions],
            ylabel = "Total cost",
            ylims = (50000, 200000)
        ),
        # Plot the power output of Generator 1
        plot(cost_of_g1, [sol.generation[1] for sol in solutions],
            ylabel = "Dispatch: G1",
            ylims = (0, 1100)
        ),
        # Plot the power output of Generator 2    
        plot(cost_of_g1, [sol.generation[2] for sol in solutions],
            ylabel = "Dispatch: G2",
            ylims = (0, 1600)
        ),
        # Plot the wind power output
        plot(cost_of_g1, [sol.wind_generation for sol in solutions],
            ylabel = "Dispatch: Wind",
            ylims = (0, 250)
        ),
        legend = false,
        xlabel = "Cost of G1"
    )
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-3976b526-54b5-4a94-bfb4-a481e113d092", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["cost_of_wind"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-left")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>26,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"value: indexString, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-center")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-right")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"indexString"=>(Observable{String} with 1 listeners. Value:
"13", nothing),"index"=>(Observable{Any} with 2 listeners. Value:
13, nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/carleton/.julia/packages/Knockout/fe7U3/src/../assets/knockout.js", "knockout_punches"=>"/Users/carleton/.julia/packages/Knockout/fe7U3/src/../assets/knockout_punches.js", "/Users/carleton/.julia/packages/InteractBase/n5zDV/src/../assets/all.js", "/Users/carleton/.julia/packages/InteractBase/n5zDV/src/../assets/style.css", "/Users/carleton/.julia/packages/InteractBulma/PH56C/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"formatted_vals\\\":[\\\"50.0\\\",\\\"55.0\\\",\\\"60.0\\\",\\\"65.0\\\",\\\"70.0\\\",\\\"75.0\\\",\\\"80.0\\\",\\\"85.0\\\",\\\"90.0\\\",\\\"95.0\\\",\\\"100.0\\\",\\\"105.0\\\",\\\"110.0\\\",\\\"115.0\\\",\\\"120.0\\\",\\\"125.0\\\",\\\"130.0\\\",\\\"135.0\\\",\\\"140.0\\\",\\\"145.0\\\",\\\"150.0\\\",\\\"155.0\\\",\\\"160.0\\\",\\\"165.0\\\",\\\"170.0\\\",\\\"175.0\\\"],\\\"changes\\\":0,\\\"indexString\\\":\\\"13\\\",\\\"index\\\":13}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        [this[\"formatted_val\"]=ko.computed(    function(){\n        return this.formatted_vals()[parseInt(this.index())-1];\n    }\n,this)]\n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-3976b526-54b5-4a94-bfb

## Nonlinear example

JuMP can also be used to solve non-linear problems (NLP). We saw a brief hint of this earlier when we used Ipopt and the `@NLobjective` macro. In the next example, we add nonlinearity to the cost of the second generator. The new cost of generation is
$$\text{generation_cost} = c_1^g\times g_1 + \frac{c_2^g\times  g_2^{1.5}}{\sqrt{1000}}.$$


In [38]:
"""
    solve_economic_dispatch(; cost_of_thermal::Vector, cost_of_wind)

Formulate and solve the economic dispatch problem given the cost of generation 
for the two thermal generators and the cost of wind generation.
"""
function solve_nonlinear_economic_dispatch(;
        cost_of_thermal = COST_GENERATION, 
        cost_of_wind = COST_WIND)
    economic_dispatch = Model(solver = IPOPT_SOLVER) 
    
    @variables(economic_dispatch, begin
        g[i=1:2] >= 0
        w >= 0
    end)

    # ===========================================================
    # You can write out nonlinear expression in the @NLobjective macro
    # The same also applies for @NLconstraint.
    @NLobjective(economic_dispatch, Min,
        cost_of_thermal[1] * g[1] + 
        cost_of_thermal[2] * g[2]^1.5 / sqrt(1000) + 
        cost_of_wind * w)
    # ===========================================================
    # Look! This bit changed.
    function generator_cost(g1, g2)
        return cost_of_thermal[1] * g1 + cost_of_thermal[2] * g2^1.5 / sqrt(1000)
    end
    JuMP.register(economic_dispatch, :generator_cost, 2, generator_cost, autodiff=true)
    @NLobjective(economic_dispatch, Min, 
        generator_cost(g[1], g[2]) + cost_of_wind * w)
    # ===========================================================
    
    for i in 1:2
        @constraint(economic_dispatch, g[i] <= GENERATION_MAX[i])
        @constraint(economic_dispatch, g[i] >= GENERATION_MIN[i])
    end    
    @constraints(economic_dispatch, begin
        w <= WIND_MAX
        sum(g[i] for i in 1:2) + w == DEMAND
    end)
    solve(economic_dispatch)
    return (
        generation = getvalue(g), 
        wind_generation = getvalue(w),
        wind_spillage = WIND_MAX - getvalue(w),
        cost = getobjectivevalue(economic_dispatch)
    )
end

solve_nonlinear_economic_dispatch

In [39]:
@manipulate for cost_of_wind in COST_WIND .* (1:0.2:3.5)
    solutions = Any[]
    cost_of_g1 = COST_GENERATION[1] .* (0.5:0.1:3.0)
    for c_g1 in cost_of_g1
        # update the incremental cost of the first generator at every iteration
        solution = solve_nonlinear_economic_dispatch(
            cost_of_thermal = [c_g1, COST_GENERATION[2]],
            cost_of_wind = cost_of_wind
        )
        push!(solutions, solution)
    end
    
    # Plot the outputs
    plot(
        # Plot the total cost
        plot(cost_of_g1, [sol.cost for sol in solutions],
            ylabel = "Total cost",
            ylims = (50000, 200000)
        ),
        # Plot the power output of Generator 1
        plot(cost_of_g1, [sol.generation[1] for sol in solutions],
            ylabel = "Dispatch: G1",
            ylims = (0, 1100)
        ),
        # Plot the power output of Generator 2    
        plot(cost_of_g1, [sol.generation[2] for sol in solutions],
            ylabel = "Dispatch: G2",
            ylims = (0, 1600)
        ),
        # Plot the wind power output
        plot(cost_of_g1, [sol.wind_generation for sol in solutions],
            ylabel = "Dispatch: Wind",
            ylims = (0, 250)
        ),
        legend = false,
        xlabel = "Cost of G1"
    )
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Scope("knockout-component-d5e4ddd4-3b54-42fd-af3a-ee391d6a7115", Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :label), Any["cost_of_wind"], Dict{Symbol,Any}(:className=>"interact ",:style=>Dict{Any,Any}(:padding=>"5px 10px 0px 10px")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-left")), 2), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :input), Any[], Dict{Symbol,Any}(:max=>13,:min=>1,:attributes=>Dict{Any,Any}(:type=>"range",Symbol("data-bind")=>"value: indexString, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient"=>"horizontal"),:step=>1,:className=>"slider slider is-fullwidth",:style=>Dict{Any,Any}()), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-center")), 1), Node{DOM}(DOM(:html, :div), Any[Node{DOM}(DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes=>Dict("data-bind"=>"text: formatted_val")), 0)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row-right")), 1)], Dict{Symbol,Any}(:attributes=>Dict("class"=>"interact-flex-row")), 7), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes"=>(Observable{Int64} with 1 listeners. Value:
0, nothing),"indexString"=>(Observable{String} with 1 listeners. Value:
"7", nothing),"index"=>(Observable{Any} with 2 listeners. Value:
7, nothing)), Set(String[]), nothing, Any["knockout"=>"/Users/carleton/.julia/packages/Knockout/fe7U3/src/../assets/knockout.js", "knockout_punches"=>"/Users/carleton/.julia/packages/Knockout/fe7U3/src/../assets/knockout_punches.js", "/Users/carleton/.julia/packages/InteractBase/n5zDV/src/../assets/all.js", "/Users/carleton/.julia/packages/InteractBase/n5zDV/src/../assets/style.css", "/Users/carleton/.julia/packages/InteractBulma/PH56C/src/../assets/main.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"formatted_vals\\\":[\\\"50.0\\\",\\\"60.0\\\",\\\"70.0\\\",\\\"80.0\\\",\\\"90.0\\\",\\\"100.0\\\",\\\"110.0\\\",\\\"120.0\\\",\\\"130.0\\\",\\\"140.0\\\",\\\"150.0\\\",\\\"160.0\\\",\\\"170.0\\\"],\\\"changes\\\":0,\\\"indexString\\\":\\\"7\\\",\\\"index\\\":7}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        [this[\"formatted_val\"]=ko.computed(    function(){\n        return this.formatted_vals()[parseInt(this.index())-1];\n    }\n,this)]\n        [this[\"changes\"].subscribe((function (val){!(this.valueFromJulia[\"changes\"]) ? (WebIO.setval({\"name\":\"changes\",\"scope\":\"knockout-component-d5e4ddd4-3b54-42fd-af3a-ee391d6a7115\",\"id\":\"ob_11\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"changes\"]=false}),self),this[\"indexString\"].subscribe((function (val){!